In [2]:
import random
import os
import re
from glob import glob
from tqdm import tqdm
import numpy as np

import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image, ImageDraw, _imaging
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import metrics
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime
from keras.callbacks import History, ModelCheckpoint
history = History()
from keras.models import model_from_json

Using TensorFlow backend.


In [29]:
def draw_it(raw_strokes):
    image = Image.new("P", (255,255)
#                       , color=1
            )
    image_draw = ImageDraw.Draw(image)

    for stroke in raw_strokes:
        for i in range(len(stroke[0])-1):

            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=255, width=6)
    # Reduce image size
    image = image.resize((32,32),Image.ANTIALIAS)
    
    return np.array(image)

In [34]:
%%time
output_height = output_width = 32
NROWS = 100000
fnames = glob('train_simplified/*.csv')
# df = pd.read_csv(fnames[0], nrows=NROWS)
# p = df.copy()
# for i in range(50):
#     df = df.append(p)

# df['drawing'] = df['drawing'].apply(eval)

# train_df = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)
for name in fnames:
# for i in tqdm(range(len(fnames))):
    train_df = pd.read_csv(fnames[0], nrows=500, skiprows=range(1,1))
    break
#     train_df = train_df.append(data)

CPU times: user 5.55 ms, sys: 5.18 ms, total: 10.7 ms
Wall time: 26.1 ms


In [35]:
# train_df[train_df['key_id']==5927670871228416]
train_df

,countrycode,drawing,key_id,recognized,timestamp,word
0,US,"[[[0, 126, 190, 255], [7, 1, 4, 0]]]",4819728134569984,True,2017-03-26 23:57:14.639540,line
1,US,"[[[0, 15, 115, 180, 251], [0, 6, 13, 27, 33]]]",6630630412517376,True,2017-03-08 19:27:39.942440,line
2,AU,"[[[32, 91, 255], [32, 48, 70]], [[179, 146, 47...",5532301360365568,True,2017-03-17 03:12:15.066140,line
3,FI,"[[[0, 9, 56, 92, 186, 255], [0, 10, 28, 36, 66...",5703119189049344,True,2017-03-17 14:13:12.645900,line
4,UA,"[[[0, 101, 155, 240, 255], [27, 11, 6, 4, 0]]]",5859843703308288,True,2017-03-12 17:11:23.741940,line
5,US,"[[[255, 157, 27, 0], [0, 71, 156, 168]]]",4952560655925248,True,2017-01-28 00:13:35.562250,line
6,TH,"[[[0, 208, 255], [3, 3, 8]], [[186, 176, 170, ...",5229145711706112,True,2017-01-25 04:19:26.770320,line
7,US,"[[[0, 71, 121, 251, 255], [16, 3, 1, 3, 0]]]",4828258107392000,True,2017-03-17 19:06:14.783300,line
8,GB,"[[[255, 225, 147, 58, 0], [12, 5, 0, 3, 10]]]",5192575570935808,True,2017-03-12 09:21:00.095530,line
9,HU,"[[[242, 227, 205, 177, 145, 0], [0, 3, 15, 38,...",6106664400846848,True,2017-03-07 10:20:05.322620,line


In [22]:
%%time
train_df = train_df.reset_index().drop('index', axis=1)
# Get only those which were correctly recognized
train_df = train_df[train_df['recognized'] == True]

CPU times: user 5.82 ms, sys: 2.53 ms, total: 8.36 ms
Wall time: 6.23 ms


In [23]:
train_df_C = train_df.copy()

In [24]:
# %%time
# train_df['drawing'] = train_df['drawing'].apply(eval)

In [25]:
def draw_it(raw_strokes, output_height = 64, output_width = 64):
    raw_strokes = eval(raw_strokes)
    image = Image.new("P", (255,255)
#                       , color=1
            )
    image_draw = ImageDraw.Draw(image)

    for stroke in raw_strokes:
        for i in range(len(stroke[0])-1):

            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=255, width=6)
    # Reduce image size
    image = image.resize((output_height,output_width),Image.ANTIALIAS)
    
    return np.array(image)


In [26]:
%%time
train_df['img'] = train_df['drawing'].apply(lambda x: draw_it(x, output_height, output_width))

CPU times: user 129 ms, sys: 7.24 ms, total: 136 ms
Wall time: 144 ms


In [28]:
train_df['img'][0][0]

array([  0,   0,   0,   0,   0, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255], dtype=uint8)

In [11]:
%%time
num_classes = train_df['word'].nunique()

CPU times: user 57.8 ms, sys: 9.86 ms, total: 67.6 ms
Wall time: 74.2 ms


In [12]:
%%time
train_df = train_df.sample(train_df.shape[0])

CPU times: user 2.07 s, sys: 311 ms, total: 2.38 s
Wall time: 2.81 s


In [13]:
%%time
img_rows = img_cols = 64
imgArr = np.vstack(train_df['img'].values).flatten().reshape((train_df['img'].shape[0], img_rows, img_cols))

CPU times: user 11.1 s, sys: 31.2 s, total: 42.2 s
Wall time: 1min 2s


In [14]:
%%time
imgArr = imgArr.reshape(imgArr.shape[0], img_rows, img_cols, 1).astype('float32')

CPU times: user 10.2 s, sys: 20.8 s, total: 30.9 s
Wall time: 42.8 s


In [15]:
%%time
y_train = train_df['word']

CPU times: user 801 µs, sys: 7.82 ms, total: 8.62 ms
Wall time: 23.4 ms


In [16]:
%%time
# Convert class labels from categorical to numerical
unique_classes_list = y_train.unique()

CPU times: user 83.9 ms, sys: 44.5 ms, total: 128 ms
Wall time: 219 ms


In [17]:
%%time
map_class_to_numeric = {k: v for v, k in enumerate(y_train.unique())}

CPU times: user 68.2 ms, sys: 12.2 ms, total: 80.5 ms
Wall time: 93.6 ms


In [18]:
%%time
map_numeric_to_class = {v: k for k, v in map_class_to_numeric.iteritems()}

CPU times: user 143 µs, sys: 1.45 ms, total: 1.6 ms
Wall time: 3.25 ms


In [19]:
%%time
y_train_numeric = y_train.apply(lambda x: map_class_to_numeric[x])

CPU times: user 565 ms, sys: 43.3 ms, total: 608 ms
Wall time: 677 ms


In [20]:
%%time
y_train_one_hot = keras.utils.to_categorical(y_train_numeric, num_classes)

CPU times: user 433 ms, sys: 615 ms, total: 1.05 s
Wall time: 1.13 s


In [21]:
%%time
num_classes = y_train_one_hot.shape[1]

CPU times: user 7 µs, sys: 3 µs, total: 10 µs
Wall time: 11 µs


In [22]:
%%time
X_train, X_test, y_train, y_test = train_test_split(imgArr, y_train_one_hot, test_size=0.2)

CPU times: user 13.1 s, sys: 42.7 s, total: 55.8 s
Wall time: 1min 21s


In [23]:
# %%time
# df['drawing'] = df['drawing'].apply(eval)

In [ ]:
%%time
pp = np.array(map(draw_it, train_df["drawing"]))
print(pp.shape)
# out_imgs = list(map(strokes_to_img, dfs["drawing"]))
# X = np.array(out_imgs)[:, :, :, :3].astype(np.float32)

In [146]:
%%time
# Convert drawing to images
df['img'] = df['drawing'].apply(lambda x: draw_it(x))
ll = np.vstack(df['img'].values).flatten().reshape((df['img'].shape[0], output_height, output_height))
print(ll.shape)
# print(ll[0])

(5100000, 32, 32)
CPU times: user 7min 43s, sys: 32.9 s, total: 8min 16s
Wall time: 8min 53s


In [147]:
# train = pd.read_csv(fnames[0], usecols=['drawing', 'recognized'], nrows=NROWS)
# train = train[train.recognized == True]
# train['drawing'] = train['drawing'].apply(eval)

In [148]:
%%time
# imheight = 32
# imwidth = 32
# # faster conversion function
# def draw_it(strokes):
#     image = Image.new("P", (256,256), color=255)
#     image_draw = ImageDraw.Draw(image)
#     for stroke in ast.literal_eval(strokes):
#         for i in range(len(stroke[0])-1):
#             image_draw.line([stroke[0][i], 
#                              stroke[1][i],
#                              stroke[0][i+1], 
#                              stroke[1][i+1]],
#                             fill=0, width=5)
#     image = image.resize((imheight, imwidth))
#     return np.array(image)/255.

#%% get train arrays
# train_grand = []
# class_paths = glob('../input/train_simplified/*.csv')
# for i,c in enumerate(tqdm(class_paths[0: num_classes])):
imagebag = bag.from_sequence(df.drawing.values).map(draw_it) 
trainarray = np.array(imagebag.compute())  # PARALLELIZE
# trainarray = np.reshape(trainarray, (ims_per_class, -1))    
# labelarray = np.full((train.shape[0], 1), i)
# trainarray = np.concatenate((labelarray, trainarray), axis=1)
# train_grand.append(trainarray)
    
# train_grand = np.array([train_grand.pop() for i in np.arange(num_classes)]) #less memory than np.concatenate
# train_grand = train_grand.reshape((-1, (imheight*imwidth+1)))

# del trainarray
# del train
print(trainarray.shape)

Process PoolWorker-64:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py", line 376, in get
    return recv()
KeyboardInterrupt
KeyboardInterrupt
    return send(obj)
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py", line 390, in put
    put((job, i, result))
  File "/usr/local/Cellar/python@

KeyboardInterrupt: 

In [77]:
# trainarray[0]

In [3]:
%%time
# output_height = output_width = 64
# NROWS = 100000
fnames = glob('train_simplified/*.csv')
fileLens = {}
fileLensRecog = {}
for i in tqdm(range(len(fnames))):
    data = pd.read_csv(fnames[i])
    len_ = data.shape[0]
    len_recog = data[data['recognized'] == True].shape[0]
    
    fileLens[i] = len_
    fileLensRecog[i] = len_recog
    


100%|██████████| 340/340 [05:32<00:00,  1.10it/s]

CPU times: user 3min 55s, sys: 38.1 s, total: 4min 33s
Wall time: 5min 32s


In [6]:
max_ = 0
min_ = 100000000
for i in tqdm(range(340)):
    val = fileLensRecog[i]
    if val > max_:
        max_ = val
    if val < min_:
        min_ = val
print(max_)
print(min_)

100%|██████████| 340/340 [00:00<00:00, 435838.44it/s]

323952
94584
